## building_matching
### Matches respective buildings between two different building footprint catalog loosely based on the so called Hungarian Method

### Initial configuration
#### To start working with this particular notebook, you need to provide necessary credential and settings
#### Below is an template of configuration, which is necessary prepare aside of this notebook and copy & paste all content in triple quotes to the next cell's input field
    """
    {
    "COS_ENDPOINT_URL": "s3.private.eu-de.cloud-object-storage.appdomain.cloud",
    "COS_AUTH_ENDPOINT_URL": "https://iam.cloud.ibm.com/oidc/token",
    "COS_APIKEY": "xxx",
    "DB2_CONNECTION_STRING": "jdbc:db2://65beb513-5d3d-4101-9001-f42e9dc954b3.brt9d04f0cmqeb8u7740.databases.appdomain.cloud:30371/BLUDB:sslConnection=true;useJDBC4ColumnNameAndLabelSemantics=false;db2.jcc.charsetDecoderEncoder=3;",
    "DB2_USERNAME": "xxx",
    "DB2_PASSWORD": "xxx",
    "UTILS_BUCKET": "notebook-utils-bucket",
    "ENERGY_ESTIMATION_BUCKET": "kenya-energy-estimation-matching",
    "JOB_STATUS_BUCKET": "notebook-job-status",
    "COUNTRY_TABLE": "FEATURES_DB_MAHARASHTRA",
    "AREA_THRESHOLD": 20
    }
    """


In [1]:
# Read notebook configuration
import getpass
import json

config_str = getpass.getpass('Enter your prepared config: ')
config = json.loads(config_str)

In [2]:
# # ! pip install pykml
# # ! pip install mgrs
# ! pip install shapely

# ! pip install ibmcloudant
# ! pip install geopandas
# ! pip install rasterio==1.3.8
# ! pip install numpy==1.23.5; python_version >= '3.7'
# ! pip install pyproj==3.6.0
# ! pip install pathos==0.3.1
# ! pip install ibmcloudant==0.4.3
# ! pip install ibm-cloud-sdk-core==3.16.7

In [3]:
# Import necessary libraries
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
from tqdm import tqdm
from pyproj import Geod
from collections import Counter
import os
from botocore.client import Config
from pyproj import Geod
import ibm_boto3
from scipy import optimize as sci_opt
import requests
import warnings
import boto3
from botocore import UNSIGNED
# import botocore.response as br
import gzip
import jaydebeapi as jdbc
import jpype
import zipfile
from io import BytesIO
import io
import shutil
import shapely

warnings.simplefilter("ignore", category=FutureWarning)

geod = Geod(ellps="WGS84")

In [4]:
# S3client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))


cos_client = ibm_boto3.client(service_name='s3',
                              ibm_api_key_id=config["COS_APIKEY"],
                              ibm_auth_endpoint=config["COS_AUTH_ENDPOINT_URL"],
                              config=Config(signature_version='oauth'),
                              endpoint_url=config["COS_ENDPOINT_URL"])


In [5]:
response = cos_client.list_objects_v2(Bucket=config["UTILS_BUCKET"])

# download utils module
try:
    from utils import *
    print('External utils succesfully imported')
    
except Exception as e:
    print('Desired packages is missing in local env, downloading it...', e)
    for obj in response['Contents']:
        name = obj['Key']
        streaming_body_1 = cos_client.get_object(Bucket=config["UTILS_BUCKET"], Key=name)['Body']
        print("Downloading to localStorage :  " + name)
        with io.FileIO(name, 'w') as file:
            for i in io.BytesIO(streaming_body_1.read()):
                file.write(i)
    from utils import *
    print('External utils succesfully imported')
  

External utils succesfully imported


In [6]:
base_path = os.getcwd()
jsons_path = os.path.join(base_path, 'tiles_jsons/')
geojsons_path = 'kenya_energy_estimated_geojsons'
sql_tablename = config["COUNTRY_TABLE"]
batch_limit = '370_000<'
trm = 'https://biospheric-vector.s3.amazonaws.com/'
area_threshold = config["AREA_THRESHOLD"]

In [8]:
matched_buildings = cos_client.list_objects_v2(Bucket=config["ENERGY_ESTIMATION_BUCKET"])
processed_ids = [int(i['Key'].split('_')[0]) for i in matched_buildings['Contents']]

In [9]:
# connect to the IBM DB2 function
def connect_to_db():

    jar = 'db2jcc4.jar'
    os.environ['CLASSPATH'] = jar

    args='-Djava.class.path=%s' % jar
    jvm_path = jpype.getDefaultJVMPath()
    try:
        jpype.startJVM(jvm_path, args)
    except Exception as e:
        print('startJVM exception: ', e)
        
    if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():
        jpype.attachThreadToJVM()
        jpype.java.lang.Thread.currentThread().setContextClassLoader(jpype.java.lang.ClassLoader.getSystemClassLoader())
        
    # create JDBC connection
    conn = jdbc.connect(
                'com.ibm.db2.jcc.DB2Driver',
                config['DB2_CONNECTION_STRING'],
                [config["DB2_USERNAME"], config["DB2_PASSWORD"]],
                'db2jcc4.jar')
    
    return conn

def fetch_builings_in_bbox(cursor, lon_min, lon_max, lat_min, lat_max):
    '''
        This particular function is aimed for obtating all entries from defined rectangle for selected SQL table
    '''

    # fetch column names from defined SQL table

    columns = ['latitude', 'longitude', 'polygon_coordinates', 'vida_confidence']
    
    # sql statement for selecting entries by defined rectangle boundaries
    sql = f"""
        SELECT {', '.join(columns)} FROM USER1.{sql_tablename}
        WHERE 
            (LATITUDE >= {lat_min}) AND 
            (LATITUDE <= {lat_max}) AND 
            (LONGITUDE >= {lon_min}) AND 
            (LONGITUDE <= {lon_max}) AND
            (AREA_IN_METERS > {area_threshold}) AND
            (FOOTPRINT_SOURCE != 'osm')
        """
    
    try:
        cursor.execute(sql)
        data = cursor.fetchall()
    except Exception as e:
        print(f"Fetch items error occured: {e}")
        print("Reconnecting to the database try again...")

        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
        
    finally:
        # reshape obtained data to the GeoDataFrame
        df = pd.DataFrame(data=data, columns=columns)
        df = gpd.GeoDataFrame(df, geometry=shapely.from_wkt(df.polygon_coordinates.astype(str)))
        df = df.drop(['polygon_coordinates'], axis=1)
        
        df.geometry = shapely.set_precision(df.geometry.array, grid_size=0.000001)
        
        df['area_in_meters'] = df["geometry"].apply(lambda g: abs(geod.geometry_area_perimeter(g)[0]))

        return df


In [13]:
dlinks_df = gpd.read_parquet('buildings_energy_estimates_links.parquet')

In [15]:
dlinks_df = dlinks_df[(dlinks_df.buildings_amount > 370_000)]
dlinks_df.index = [i for i in range(len(dlinks_df))]
dlinks_df

,grid_cell_id,download_link,geometry,buildings_amount
0,525390,https://biospheric-vector.s3.amazonaws.com/ope...,"POLYGON ((36.50000 -1.00000, 36.75000 -1.00000...",389994.0
1,525391,https://biospheric-vector.s3.amazonaws.com/ope...,"POLYGON ((36.75000 -1.00000, 37.00000 -1.00000...",676605.0
2,526832,https://biospheric-vector.s3.amazonaws.com/ope...,"POLYGON ((36.75000 -1.25000, 37.00000 -1.25000...",651071.0
3,523942,https://biospheric-vector.s3.amazonaws.com/ope...,"POLYGON ((34.75000 -0.75000, 35.00000 -0.75000...",443147.0
4,522501,https://biospheric-vector.s3.amazonaws.com/ope...,"POLYGON ((34.75000 -0.50000, 35.00000 -0.50000...",606364.0
5,518177,https://biospheric-vector.s3.amazonaws.com/ope...,"POLYGON ((34.50000 0.25000, 34.75000 0.25000, ...",445679.0


In [16]:

dlinks_df = dlinks_df[~dlinks_df.grid_cell_id.isin(processed_ids)]
dlinks_df = dlinks_df.sort_values(by='buildings_amount', ascending=True)
dlinks_df

,grid_cell_id,download_link,geometry,buildings_amount
1,525391,https://biospheric-vector.s3.amazonaws.com/ope...,"POLYGON ((36.75000 -1.00000, 37.00000 -1.00000...",676605.0


In [17]:
dlinks_df.buildings_amount.sum()

676605.0

In [20]:

def extract_geojson(obj_key):
     try:
        obj = s3.Object('biospheric-vector', obj_key)
        obj_bytes = obj.get()['Body'].read()

        json_filename = ''
        with io.BytesIO(obj_bytes) as tf:
        # rewind the file
            tf.seek(0)
               # Read the file as a zipfile and process the members
            with zipfile.ZipFile(tf, mode='r') as zipf:
                for subfile in zipf.namelist():
                        if '.geojson' in subfile:
                            json_filename = subfile
                            zipf.extract(subfile, path=geojsons_path)

        return os.path.join(geojsons_path, json_filename)
     except Exception as e:
        print(obj_key, e)
        
        
def read_geojson(geojson_path):
    
    df = gpd.read_file(geojson_path)
    
    df['longitude_2'] = df['geometry'].apply(lambda g: g.centroid.xy[0][0])
    df['latitude_2'] = df['geometry'].apply(lambda g: g.centroid.xy[1][0])
    
    df.geometry = shapely.set_precision(df.geometry.array, grid_size=0.000001)
    
    return df

In [21]:
# generate coordinate grid with defined tilesize and overlap
def generate_grid(
                    country_bbox: list,
                    tile_bbox: list,
                    overlap=0.000
                ):
    
    row_col_dim = tile_bbox
    
    rows_cols = [
      int(abs(country_bbox[0][0] - country_bbox[0][1]) // row_col_dim[0]) if abs(country_bbox[0][0] - country_bbox[0][1]) % row_col_dim[0] == 0 else int(abs(country_bbox[0][0] - country_bbox[0][1]) // row_col_dim[0]) + 1,
      int(abs(country_bbox[1][0] - country_bbox[1][1]) // row_col_dim[1]) if abs(country_bbox[1][0] - country_bbox[1][1]) % row_col_dim[1] == 0 else int(abs(country_bbox[1][0] - country_bbox[1][1]) // row_col_dim[1]) + 1
    ]
    
    columns_amount = rows_cols[0]
    rows_amount = rows_cols[1]
    
    tile_width = row_col_dim[0]
    tile_height = row_col_dim[1]

    tiff_height = abs(country_bbox[1][0] - country_bbox[1][1])
    tiff_width = abs(country_bbox[0][0] - country_bbox[0][1])
    
    images_coords = []
    
    for col_idx in range(1, columns_amount + 1):
    
        row_start = country_bbox[0][0] + max(tile_width * (col_idx - 1) - overlap, 0)

        if col_idx != columns_amount:

            row_limits = [row_start, country_bbox[0][0] + (tile_width * col_idx)]
        elif col_idx == columns_amount:
            row_limits = [row_start, country_bbox[0][0] + tiff_width]

        for row_idx in range(1, rows_amount + 1):

            col_start = country_bbox[1][0] + max(tile_height * (row_idx - 1) - overlap, 0)

            if row_idx != rows_amount:
                col_limits = [col_start, country_bbox[1][0] + (tile_height * row_idx)]
            elif row_idx == rows_amount:
                col_limits = [col_start, country_bbox[1][0] + tiff_height]

            coords = [row_limits, col_limits]
            
            images_coords.append(coords)

    return images_coords


# calculate ratio width height ratio
def get_bbox_wh_ratio(geometry):
    w1, h1, w2, h2 = geometry.bounds
    w1 = round(w2 - w1, 8)
    h1 = round(h2 - h1, 8)
    return w1/h1

# calculate distance between buildings
def get_distance_between_buildings(building1, building2):
    w = building1[0][0] - building2[0][0]
    h = building1[1][0] - building2[1][0]

    lat_lon_koef = (0.0008988810840833139 + 0.0009006895240588619)/200

    distance = np.sqrt((w**2 + h**2)) / lat_lon_koef
    if distance < 1:
        return 1
    
    return distance


# calculate matching ratios and combine them
def calculate_ratios(vida_matched, building, items_treshold):

    vida_matched['area_similarity'] = vida_matched['area_in_meters'].apply(lambda a: abs(1 - a/building.area_in_meters))

    w1, h1, w2, h2 = building.geometry.bounds
    w1 = round(w2 - w1, 8)
    h1 = round(h2 - h1, 8)
    bwhr = w1/h1

    building1 = building.geometry.centroid.xy

    vida_matched['shape_similarity'] = vida_matched['geometry'].apply(lambda g: abs(bwhr - get_bbox_wh_ratio(g)))

    vida_matched['distance_similarity'] = vida_matched['geometry'].apply(lambda g: abs(1 - 1/get_distance_between_buildings(building1, g.centroid.xy)))

    vida_matched['combined_ratios'] = vida_matched['area_similarity'] + vida_matched['shape_similarity'] + vida_matched['distance_similarity']
    
    vida_matched = vida_matched.sort_values(by='combined_ratios', ascending=True)

    # vida_matched = vida_matched.head(items_treshold)
    
    vida_matched['base_geometry'] = [building.geometry for _ in range(len(vida_matched))]
    
    vida_matched = vida_matched.drop(
        [
            'origin_id', 
            'elec access (%)', 
            'cons (kWh/month)', 
            'std cons (kWh/month)',
            'a_alls_elec', 
            'b_alls_elec', 
            'a_alls_dem', 
            'b_alls_dem',
            'area_in_meters', 
            'area_similarity', 
            'shape_similarity', 
            'distance_similarity',
        ],
        axis=1
    )

    return vida_matched

def prepare_buildings(vida_df, osm_df, bbox):
    
    vida_filtered = vida_df[
            (vida_df.longitude_2 >= bbox[0][0]) &
            (vida_df.longitude_2 <= bbox[0][1]) &
            (vida_df.latitude_2 >= bbox[1][0]) &
            (vida_df.latitude_2 <= bbox[1][1])
    ].drop(['longitude_2', 'latitude_2'], axis=1).copy()
    
    
    osm_filtered = osm_df[
            (osm_df.longitude >= bbox[0][0]) &
            (osm_df.longitude <= bbox[0][1]) &
            (osm_df.latitude >= bbox[1][0]) &
            (osm_df.latitude <= bbox[1][1])
    ].copy()

    if len(osm_filtered) == 0 or len(vida_filtered) == 0:
        
        pass

    else:
        # 0,0009
        
        matched_buildings = []
        items_limit = min(len(osm_filtered), len(vida_filtered))
#         print(len(osm_filtered), len(vida_filtered))
        for idx, building in enumerate(osm_filtered.itertuples()):
            try:

                max_intersection_row = calculate_ratios(vida_filtered, building, items_limit)
                matched_buildings.append(max_intersection_row)
            except Exception as e:
                print(f'Ratios calculaiton err: {e}')
        
        try:
            df = pd.concat(matched_buildings)
#             print(f"Buildings matched: {len(df)} of {len(osm_filtered)}")
            return df
        
        except Exception as e:
            print(e)
            return None


In [22]:
# algorithm for one to one building matching
def match_buildings(main_df, tile_idx):

    # assing indexes ang geometry ids
    main_df.index = [i  for i in range(len(main_df))]
    main_df['geometry_id'] = main_df['base_geometry'].apply(lambda g: str(g))
    main_df['secondary_geometry_id'] = main_df['geometry'].apply(lambda g: str(g))
    main_df = main_df.drop(['base_geometry', 'geometry'], axis=1)

    # main_df.groupby(['geometry_id']).agg({'base_height': max})
    vida_keys = {k: v for v, k in enumerate(list(set(main_df['geometry_id'])))}
    main_df['g_key'] = main_df['geometry_id'].apply(lambda k: vida_keys[k])


    # cost matrix generation
    cost_matrix = []

    for poly_id in vida_keys.values():

        sub_df = main_df[main_df['g_key'] == poly_id].sort_values(by='combined_ratios')
        sub_df = sub_df.drop_duplicates(subset='secondary_geometry_id')
        cost_matrix.append(list(sub_df.combined_ratios))

    cost_matrix = np.array(cost_matrix)

    row_idxs, col_idxs = sci_opt.linear_sum_assignment(cost_matrix, maximize=False) 

    # matches processing
    matches_row_idxs = {}
    vida_keys_s = {k:v for v, k in vida_keys.items()}

    for row_idx, col_idx in zip(row_idxs, col_idxs):

        sub_df = main_df[main_df['g_key'] == row_idx].sort_values(by='combined_ratios')

        matched_building = main_df.secondary_geometry_id.iloc[col_idx]
        # matches[vida_keys_s[row_idx]] = matched_building
        matches_row_idxs[vida_keys_s[row_idx]] = [sub_df.combined_ratios.iloc[col_idx], matched_building]

    # save all the matches to the json file for further processing
    filename = f'matches_row_idxs_{tile_idx}.json'
    file_path = f'{jsons_path}/{filename}'
    with open(file_path, "w") as outfile: 
        json.dump(matches_row_idxs, outfile)

In [23]:
conn = connect_to_db()
cursor = conn.cursor()

/var/folders/42/bkr77x_n1qv118pt28lzd3380000gn/T/ipykernel_80338/3470458869.py:15: DeprecationWarning: jpype._core.isThreadAttachedToJVM is deprecated, use java.lang.Thread.isAttached instead
  if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():


In [24]:
processing_state = {
    "total_unique_matched_count": 0,
    "total_matched_count": 0
}

In [25]:
def log_state_to_bucket(processing_state: dict):
    '''
        Function for updating matching state to the 'notebook-job-status' bucket
        Each call of this function uploads a selected json file with updated state to the afore mentioned bucket
    '''
    
    filename = f'matching_buildings_Kenya_status_{batch_limit}.json'
    with open(filename, "w") as outfile:
                json.dump(processing_state, outfile)
                
    cos_client.upload_file(
        Filename=filename,
        Bucket=config["JOB_STATUS_BUCKET"],
        Key=filename,
        )

In [ ]:
total_unique_matched_count = 0
total_matched_count = 0

total_grid_cells = len(dlinks_df)

for grid_cell_idx, row in enumerate(tqdm(dlinks_df.itertuples(), desc='Matching buildings:')):
    
    print(f'Processing cell {row.grid_cell_id}')
    os.makedirs(jsons_path, exist_ok = True)
    
    obj_key = row.download_link.replace(trm,'')
    
    json_path = extract_geojson(obj_key)
    df_energy_estimations = read_geojson(json_path)
    
    print(f'Buildings with energy estimation: {len(df_energy_estimations)}')
    
    min_lon, min_lat, max_lon, max_lat = row.geometry.bounds
    df_raw = fetch_builings_in_bbox(cursor, min_lon, max_lon, min_lat, max_lat)
    
    print(f'DB2 buildings: {len(df_raw)}')
    
    # Kenya bounding box
    country_bbox = [
        [min_lon, max_lon],
        [min_lat, max_lat]
    ]

    # inside tile bounding box size in lat lon degrees
    
    if len(df_energy_estimations) < 700_000:
        tile_bbox = [.015, .015]
    else:
        tile_bbox = [.007, .007]

    # generate tiles for country
    all_country_tiles = generate_grid(country_bbox, tile_bbox, overlap=0.0025)  
    
    print('grid generated')
    # iterate through generated tiles
    for tile_idx, bbox in tqdm(enumerate(all_country_tiles), desc='Calculating buildings in tiles', total=len(all_country_tiles)):
        # start from specific tile
        if tile_idx >= 0:
#             print(f'Processing tile {tile_idx} of {len(all_country_tiles)} {round(100*tile_idx/len(all_country_tiles), 2)}%', end='\r')
            filtered_df = prepare_buildings(df_energy_estimations, df_raw, bbox)
            if isinstance(filtered_df, pd.DataFrame):
                
                match_buildings(filtered_df, tile_idx)
                
    all_files = os.listdir(jsons_path)
    matches_row_idxs_jsons = [i for i in all_files if 'matches_row_idxs' in i]
    matches_row_idxs_jsons.sort()

    all_matches = {}

    for i in matches_row_idxs_jsons:

        matched = json.load(open(f'{jsons_path}/{i}'))

        for k, v in matched.items():
            if k in all_matches.keys():
                all_matches[k].append([v])
            else:
                all_matches[k] = [[v]]
                
                
    shutil.rmtree(jsons_path)
                
    nonunique_matches = [k for k, v in all_matches.items() if len(v) > 1]
    unique_matches = [k for k, v in all_matches.items() if len(v) == 1]
#     print(f' nonunique_matches {len(nonunique_matches)} unique_matches {len(unique_matches)}')

    filtered_matches = {}
    for idx, k in enumerate(nonunique_matches):

        ratios = []

        for i in all_matches[k]:
            try:
                ratios.append(i[0][0])
            except Exception as e:
                print(e)

        filtered_matches[k] = all_matches[k][ratios.index(min(ratios))][0][1]
        

    df_raw['geometry_id'] = df_raw['geometry'].apply(lambda g: str(g))
    
    mapping_dict = {k: all_matches[k][0][0][1] for k in unique_matches}
    print("Unique matches amount", len(mapping_dict))

    total_unique_matched_count += len(mapping_dict)
    # concat unique matches + filtered matches
    mapping_dict = mapping_dict | filtered_matches
    print("All matches amount", len(mapping_dict))

    total_matched_count += len(mapping_dict)
    # filter main df only for matched items
    
    main_df = df_raw[df_raw.geometry_id.isin([i for i in mapping_dict.keys()])].copy()
    main_df['secondary_geometry_id'] = main_df['geometry_id'].apply(lambda building_id: mapping_dict[building_id])
    
    df_energy_estimations['secondary_geometry_id'] = df_energy_estimations['geometry'].astype(str)

    df_energy_estimations = df_energy_estimations[[
                        'elec access (%)', 
                        'cons (kWh/month)', 
                        'std cons (kWh/month)', 
                        'a_alls_elec', 
                        'b_alls_elec', 
                        'a_alls_dem', 
                        'b_alls_dem',
                        'secondary_geometry_id'
                    ]].copy()
    
    main_df = pd.merge(main_df, df_energy_estimations, on='secondary_geometry_id')
    
    main_df = main_df[['latitude', 'longitude', 'vida_confidence', 'geometry', 'area_in_meters', 'elec access (%)', 'cons (kWh/month)', 'std cons (kWh/month)', 'a_alls_elec', 'b_alls_elec', 'a_alls_dem', 'b_alls_dem']]
    
    parquet_filename = f'{row.grid_cell_id}_matched_buildings.parquet'
    
    main_df.drop_duplicates().to_parquet(parquet_filename)
    
    cos_client.upload_file(Filename=parquet_filename,Bucket=config["ENERGY_ESTIMATION_BUCKET"],Key=parquet_filename)
    
    processing_state['processed_counts'] = f'Processed: {grid_cell_idx} of {total_grid_cells} | {round(100*grid_cell_idx/total_grid_cells, 3)}%'
    processing_state['processed_grid_cell_id'] = row.grid_cell_id
    processing_state['total_unique_matched_count'] = total_unique_matched_count
    processing_state['total_matched_count'] = total_matched_count
    
    log_state_to_bucket(processing_state)
    
    print()
